In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
import torch
from huggingface_hub import login

login()



In [4]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map=device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [21]:
from utils.dataset import ClaimVerificationDataset

train_dataset = ClaimVerificationDataset(f"data/train-eng.csv")
test_dataset = ClaimVerificationDataset(f"data/dev-eng.csv")

print(f"Train dataset length: {len(train_dataset)}")
print(f"Test dataset length: {len(test_dataset)}")

Train dataset length: 11374
Test dataset length: 1171


In [28]:
conversation = [
            {"role": "system", "content": "You are an AI assistant designed to extract claims from a given passage of text. Keep it short and return the claim in the text. Only return the big idea and exclude unneeded details."},
            {"role": "user", "content": train_dataset[0]["text"]},
            {"role": "assistant", "content": train_dataset[0]["claim"]},
            {"role": "user", "content": train_dataset[1]["text"]},
            {"role": "assistant", "content": train_dataset[1]["claim"]},
            {"role": "user", "content": train_dataset[15]["text"]},
            {"role": "assistant", "content": train_dataset[15]["claim"]},
            {"role": "user", "content": train_dataset[23]["text"]},
            {"role": "assistant", "content": train_dataset[23]["claim"]},
            {"role": "user", "content": train_dataset[29]["text"]},
            {"role": "assistant", "content": train_dataset[29]["claim"]},
        ]

In [29]:
new_passage = "The government has announced a new policy to reduce carbon emissions by 40% before 2030. This is actually rediculous I aint paying for that????"

# Append new passage as user input
conversation.append({"role": "user", "content": new_passage})

In [30]:
formatted_prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)

In [ ]:
generation_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.1,
    top_p=0.8,
    return_full_text=False
)

output = generation_pipeline(formatted_prompt)
claim = output[0]['generated_text'].split(".")[0] + "."
print(claim)

Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Government aims to reduce carbon emissions by 40% by 2030.
